# lstm分类模型搭建

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#重置图模型
tf.reset_default_graph() 

In [3]:
learning_rate = 0.1
state_size = 128 # hidden layer num of features
n_classes = 19
n_features = 23

In [4]:
x = tf.placeholder(tf.float32, [None, None, n_features], name='input_placeholder')  #batch_size, time_step, feat_len
y = tf.placeholder(tf.float32, [None, None, n_classes], name='labels_placeholder')  #batch_size, time_step, n_classes

batch_size = tf.placeholder(tf.int32, (), name='batch_size')
time_steps = tf.placeholder(tf.int32, (), name='times_step')

In [5]:
rnn_cell = tf.contrib.rnn.BasicLSTMCell(state_size)
init_state = rnn_cell.zero_state(batch_size, dtype=tf.float32)  #状态初始化为0


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


In [6]:
weights = tf.Variable(tf.truncated_normal([state_size, n_classes], stddev=0.1))
biases = tf.Variable(tf.zeros(n_classes))

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
outputs, final_state = tf.nn.dynamic_rnn(cell=rnn_cell, 
                                         inputs=x,
                                        initial_state=init_state,
                                        dtype=tf.float32)
#outputs是[batch_size, max_time, state_size], 包含每次时间步的输出
#final_state是最后一次的状态[batch_size, state_size]

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


In [8]:
#计算rnn输出的状态值，先reshape成2维以便可以和W，b进行运算
#再reshape回来，变成batch_size * time_step * n_classes
state_out = tf.matmul(tf.reshape(outputs, [-1, state_size]), weights) + biases
logits = tf.reshape(state_out, [batch_size, time_steps, n_classes])

In [9]:
#进行softmax计算
probs = tf.nn.softmax(logits, -1)   #-1也是默认值，表示在最后一维进行运算
predict = tf.argmax(probs, -1)  #最大的概率在最后一维的哪一列，从0计数，
                                #维度变为  batch_size * time_step

In [10]:
true_val = tf.argmax(y, -1)   #真实结果，维度为 batch_size * time_step
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            logits=logits, labels=y))  #计算交叉熵

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [11]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)  #优化方法
train_op = optimizer.minimize(loss_op)